## Content Based Router (from YAML spec)

This notebook demonstrates using `FunctionalTransformer` with a `RoutingRule` to redirect a message based on the message content.

The Message is routed based on the partial `MessageRoutable` returned from JSONata expression, combining it with default values.

These are the fields in MessageRoutable:

- **topics** (Union[str, List[str]]): The topics to which the message should be published.
- **recipient_list** (List[AgentTag]): List of agents tagged in the message.
- **payload** (JsonDict): The actual content or payload of the message.
- **format** (str): The type of the message.
- **forward_header** (Optional[ForwardHeader]): The header for a forwarded message.
- **context** (Optional[JsonDict]): The context of the message.
- **enrich_with_history** (Optional[int]): The number of previous messages to include in the context.

In this example, we route if based on `routing_key` in the payload.


[ProbeAgent] --(`{"routing_key": "topic_1"}`)-> (default_topic) -> [Routing Agent/RoutingAgent:wire_message] -> (**topic_1**)

[ProbeAgent] --(`{"routing_key": "topic_2"}`)-> (default_topic) -> [Routing Agent/RoutingAgent:wire_message] -> (**topic_2**)


In [1]:
from pydantic import BaseModel

class CBRMessage(BaseModel):
    routing_key: str
    body: str

In [2]:
from rustic_ai.core.guild.builders import AgentBuilder, GuildBuilder
from rustic_ai.core.guild.dsl import GuildTopics

guild_default_topic = GuildTopics.DEFAULT_TOPICS[0]

# Create the Guild with the Routing Agent and the Routing Rules
guild = GuildBuilder.from_yaml_file("./003_content_based_router.yaml").launch("myorg")

In [3]:
from rustic_ai.core.agents.testutils.probe_agent import ProbeAgent

probe_agent = (
    AgentBuilder(ProbeAgent)
    .set_id("ProbeAgent")
    .set_name("Probe Agent")
    .set_description("A probe agent to test the routing of messages.")
    .add_additional_topic("topic_1")
    .add_additional_topic("topic_2")
    .build()
)

guild._add_local_agent(probe_agent)

In [4]:
probe_agent.publish_with_guild_route(
    payload=CBRMessage(routing_key="topic_1", body="This is a message for topic 1"),
    topic=guild_default_topic,
)

In [5]:
probe_agent.print_message_history()

(default_topic) -> [Routing Agent/RoutingAgent:wire_message] -> (topic_1)


In [6]:
probe_agent.publish_with_guild_route(
    payload=CBRMessage(routing_key="topic_2", body="This is a message for topic 1"),
    topic=guild_default_topic,
)

In [7]:
probe_agent.print_message_history()

(default_topic) -> [Routing Agent/RoutingAgent:wire_message] -> (topic_2)
